In [8]:
import sys
import os
from bit.transaction import sign_tx
import pandas as pd
from coincurve import keys
from dotenv import load_dotenv
import subprocess
import json
from bit import wif_to_key

from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3 import Web3, middleware, Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware
from eth_account import Account
from getpass import getpass

import requests

In [9]:
# connect to Web3
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

# set gas price strategy to built-in "medium" algorithm (est ~5min per tx)
# see https://web3py.readthedocs.io/en/stable/gas_price.html?highlight=gas
# see https://ethgasstation.info/ API for a more accurate strategy
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)

# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

# Set constants
BTC = 'btc'
ETH = 'eth'
BTCTEST = 'btc-test'

In [10]:
# Create function to derive wallets:
def derive_wallets(mnemonic,coin,depth):
    command = './derive -g --mnemonic=mnemonic --cols=all --coin=ETH --numderive=3 --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status=p.wait()
    keys = json.loads(output)
    return keys

# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {ETH,BTC}
key = {}
for coin in coins:
    key[coin]=derive_wallets(mnemonic,coin,3)

# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin,priv_key):
    # print(coin)
    # print(priv_key)
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTC:
        key = wif_to_key("")
        return key.PrivateKeyTestnet(priv_key)

print(key)


{'btc': [{'path': "m/44'/60'/0'/0/0", 'address': '0x3De0A2fD4A90f9A160ebb2B8711192D1F0eB339D', 'xprv': 'xprvA3W3uzhjas8mpZrqS7oW9ar2SWxLGhwcPFT7S2j7VM8e9PbaYcfSJo4aDdsjd7yi9iPcKkvxg7TzmeNJniz84Y4rYgTaarwiMk66gZNwj3P', 'xpub': 'xpub6GVQKWEdREh533wJY9LWWinkzYnpgAfTkUNiER8j3gfd2Bvj69ygrbP44uSWSEu9BwzQ2mboHQp17puXxT3y9xnXCt256vtjkzV2caX1xeT', 'privkey': '0x82d63adc8be924a92ef77c6e3286c4e80eaebad63ba2b91c27801a62ba2f9410', 'pubkey': '02c93d99b9686cabed081f1eb262b81a435ae2816ba940e0fea550a36603d8077b', 'pubkeyhash': 'a90e4da9a141168d3e0ac3821015190a79df729c', 'index': 0}, {'path': "m/44'/60'/0'/0/1", 'address': '0xc5557310a53b90f18E926431B1896c29Df0dcc1a', 'xprv': 'xprvA3W3uzhjas8msDSBJH94x7mqj53hfhCweGzWNRqsHamAi3MrZh4cDT4VywH5UH7UboqUbVazKeEn9gLy7TgeMbtWSuyYk852kBG75GZmB4r', 'xpub': 'xpub6GVQKWEdREh55hWeQJg5KFiaH6tC59vo1Vv7ApFUqvJ9aqh17ENrmFNyqBeWXxBLL8hxPeeUw4A8HfovbNetZcGkNj6p1VoJiDt4UJ3L9RU', 'privkey': '0xf0c147a367c5732f750e0dc072540f71ec3e3a87fafee9e7516982861e9e259e', 'pubkey': '029

In [11]:

# Create a variable to hold keys
eth_owner = priv_key_to_account(ETH,key['eth'][0]['privkey'])
print(eth_owner)

# # Create a variable to hold coins
eth_recipient = key['eth'][2]['address']
eth_recipient_pubkey = key['eth'][2]['pubkey']
print(eth_recipient)

0x926841419b49910D80fCEa75d587BE3aefFf560e


In [12]:
print(key['btc'])
print("*******")
print(key['eth'])

btc_owner = priv_key_to_account(BTC,key['btc'][2]['xprv']) # unable to get this to run without error
btc_recipient = key['btc'][2]['address']
btc_recipient_pubkey = key['btc'][2]['pubkey']


[{'path': "m/44'/60'/0'/0/0", 'address': '0x3De0A2fD4A90f9A160ebb2B8711192D1F0eB339D', 'xprv': 'xprvA3W3uzhjas8mpZrqS7oW9ar2SWxLGhwcPFT7S2j7VM8e9PbaYcfSJo4aDdsjd7yi9iPcKkvxg7TzmeNJniz84Y4rYgTaarwiMk66gZNwj3P', 'xpub': 'xpub6GVQKWEdREh533wJY9LWWinkzYnpgAfTkUNiER8j3gfd2Bvj69ygrbP44uSWSEu9BwzQ2mboHQp17puXxT3y9xnXCt256vtjkzV2caX1xeT', 'privkey': '0x82d63adc8be924a92ef77c6e3286c4e80eaebad63ba2b91c27801a62ba2f9410', 'pubkey': '02c93d99b9686cabed081f1eb262b81a435ae2816ba940e0fea550a36603d8077b', 'pubkeyhash': 'a90e4da9a141168d3e0ac3821015190a79df729c', 'index': 0}, {'path': "m/44'/60'/0'/0/1", 'address': '0xc5557310a53b90f18E926431B1896c29Df0dcc1a', 'xprv': 'xprvA3W3uzhjas8msDSBJH94x7mqj53hfhCweGzWNRqsHamAi3MrZh4cDT4VywH5UH7UboqUbVazKeEn9gLy7TgeMbtWSuyYk852kBG75GZmB4r', 'xpub': 'xpub6GVQKWEdREh55hWeQJg5KFiaH6tC59vo1Vv7ApFUqvJ9aqh17ENrmFNyqBeWXxBLL8hxPeeUw4A8HfovbNetZcGkNj6p1VoJiDt4UJ3L9RU', 'privkey': '0xf0c147a367c5732f750e0dc072540f71ec3e3a87fafee9e7516982861e9e259e', 'pubkey': '0299b67384c

ValueError: Decoded checksum b'\x00' derived from "" is not equal to hash checksum b']\xf6\xe0\xe2'.

In [5]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, recipient, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account, "to": recipient, "value": amount})
        return {
            "from": account,
            "to": recipient, 
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(w3.toChecksumAddress(account.address))}
    elif coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(btc_acc,[(recipient,amount,BTC)])

# # Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, recipient, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account, recipient,amount)
        signed_tx = account.sign_transaction(raw_tx)
        result = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
        print (result.hex())
        return result.hex()
    elif coin == BTCTEST:
        raw_tx = create_tx(coin, account, recipient,amount)
        signed_tx = account.bit(sign_tx(raw_tx))
        result = account.bit(send_tx(signed_tx))
        print (result.hex())
        return result.hex()

txHash = send_tx(ETH, eth_owner, eth_recipient, w3.toWei(12345,'ether')) 
print(txHash)

# txHash = send_tx(BTC, btc_owner, btc_recipient, w3.toWei(12345,'ether')) 
# print(txHash)

TypeError: Address <eth_account.signers.local.LocalAccount object at 0x7fbf7f262880> must be provided as a string